In [56]:
# -*- coding:utf-8 -*-
import os
import warnings

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller as ADF

In [57]:
# 预设值
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None  # 展示所有列

In [58]:
RAW_TRAIN = './datasets/serverless/train.csv'

# 加载原始数据
train_df = pd.read_csv(RAW_TRAIN)
train_df = train_df.sort_values(by=['QUEUE_ID', 'DOTTING_TIME']).reset_index(drop=True)

In [61]:
def stationarityTest(data):
    """
    平稳性检验
    :return:
    """
    predictnum = 5

    data = data.iloc[: len(data) - predictnum]

    diff = 0
    adf = ADF(data['CWXT_DB:184:D:\\'])
    while adf[1] > 0.05:
        diff = diff + 1
        adf = ADF(data['CWXT_DB:184:D:\\'].diff(diff).dropna())

    print(u'原始序列经过%s阶差分后归于平稳，p值为%s' % (diff, adf[1]))


def whitenoiseTest(data, queueId: int):
    """
    白噪声检验
    :return:
    """
    res_df = acorr_ljungbox(data, lags=[1])
    p = res_df['lb_pvalue'].values[0]
    if p < 0.05:
        print(u'{} 原始序列为非白噪声序列，对应的p值为：{}'.format(queueId, p))
    else:
        print(u'{} 原始该序列为白噪声序列，对应的p值为：{}'.format(queueId, p))
        return True
    res_df = acorr_ljungbox(data.diff().dropna(), lags=[10])
    p = res_df['lb_pvalue'].values[0]
    if p < 0.05:
        print(u'{} 一阶差分序列为非白噪声序列，对应的p值为：{}'.format(queueId, p))
    else:
        print(u'{} 一阶差分该序列为白噪声序列，对应的p值为：{}'.format(queueId, p))
    return False


In [ ]:
"""
自相关函数 (ACF)。延迟为 k 时，这是相距 k 个时间间隔的序列值之间的相关性。
偏自相关函数 (PACF)。延迟为 k 时，这是相距 k 个时间间隔的序列值之间的相关性，同时考虑了间隔之间的值。
截尾与拖尾是什么鬼？
截尾是指时间序列的自相关函数（ACF）或偏自相关函数（PACF）在某阶后均为0的性质（比如AR的PACF）；拖尾是ACF或PACF并不在某阶后均为0的性质（比如AR的ACF）。

截尾：在大于某个常数k后快速趋于0为k阶截尾
拖尾：始终有非零取值，不会在k大于某个常数后就恒等于零(或在0附近随机波动)
"""
queueIds_df = pd.read_csv('./datasets/serverless/q_ids.csv')
queueIds: np.ndarray = queueIds_df['QUEUE_ID'].values
for i, queueId in np.ndenumerate(queueIds):
    # i是只有一个下标index的tuple，访问时直接i[0]
    raw = queueIds_df.iloc[i[0]]
    start = raw['ranges_start']
    end = raw['ranges_end']
    queue_df = train_df[train_df.QUEUE_ID == queueId]
    cpu_s = queue_df['CPU_USAGE']
    plot_acf(cpu_s, title='Autocorrelation ID_{}'.format(queueId), lags=100)
    plt.show()

In [ ]:
queueIds_df = pd.read_csv('./datasets/serverless/q_ids.csv')
queueIds: np.ndarray = queueIds_df['QUEUE_ID'].values
for i, queueId in np.ndenumerate(queueIds):
    # i是只有一个下标index的tuple，访问时直接i[0]
    raw = queueIds_df.iloc[i[0]]
    start = raw['ranges_start']
    end = raw['ranges_end']
    queue_df = train_df[train_df.QUEUE_ID == queueId]
    cpu_s = queue_df['CPU_USAGE']
    plot_pacf(cpu_s, title='Autocorrelation ID_{}'.format(queueId), lags=100)
    plt.show()

In [62]:
queueIds_df = pd.read_csv('./datasets/serverless/q_ids.csv')
queueIds: np.ndarray = queueIds_df['QUEUE_ID'].values

error_queueIds = []
for i, queueId in np.ndenumerate(queueIds):
    # i是只有一个下标index的tuple，访问时直接i[0]
    raw = queueIds_df.iloc[i[0]]
    start = raw['ranges_start']
    end = raw['ranges_end']
    queue_df = train_df[train_df.QUEUE_ID == queueId]
    cpu_s = queue_df['CPU_USAGE']
    print("========================")
    isWhite = whitenoiseTest(cpu_s, queueId)
    if isWhite:
        error_queueIds.append(queueId)

print(error_queueIds)

2 原始序列为非白噪声序列，对应的p值为：0.0
2 一阶差分序列为非白噪声序列，对应的p值为：0.0
3 原始序列为非白噪声序列，对应的p值为：2.1874005862757105e-16
3 一阶差分序列为非白噪声序列，对应的p值为：0.0
4 原始序列为非白噪声序列，对应的p值为：6.545771145197786e-32
4 一阶差分序列为非白噪声序列，对应的p值为：0.0
27 原始序列为非白噪声序列，对应的p值为：1.3025571680381248e-45
27 一阶差分序列为非白噪声序列，对应的p值为：0.0
36 原始序列为非白噪声序列，对应的p值为：0.0
36 一阶差分序列为非白噪声序列，对应的p值为：4.711706031377924e-89
233 原始序列为非白噪声序列，对应的p值为：0.0
233 一阶差分序列为非白噪声序列，对应的p值为：8.37636394300812e-53
287 原始序列为非白噪声序列，对应的p值为：0.0
287 一阶差分序列为非白噪声序列，对应的p值为：1.685496747387129e-92
291 原始序列为非白噪声序列，对应的p值为：0.0
291 一阶差分序列为非白噪声序列，对应的p值为：0.0007805292195434946
293 原始序列为非白噪声序列，对应的p值为：0.0
293 一阶差分序列为非白噪声序列，对应的p值为：1.2958077827459848e-09
21825 原始序列为非白噪声序列，对应的p值为：9.897844206305218e-116
21825 一阶差分序列为非白噪声序列，对应的p值为：0.0
82695 原始序列为非白噪声序列，对应的p值为：5.109442090487695e-44
82695 一阶差分序列为非白噪声序列，对应的p值为：0.0
82697 原始序列为非白噪声序列，对应的p值为：6.395696574683256e-35
82697 一阶差分序列为非白噪声序列，对应的p值为：0.0
82929 原始序列为非白噪声序列，对应的p值为：2.41930526958731e-286
82929 一阶差分序列为非白噪声序列，对应的p值为：5.346188614641333e-289
83109 原始序列为非白噪声序列，对应的p值为：6.8763185